In [1]:
!pip install qiskit
!pip install qiskit_aer                   #simulacion de alto desempeño
!pip install "qiskit[visualization]"      #visualizacion de los circitos cuanticos
!pip install qiskit_machine_learning
!pip install qiskit_algorithms
!pip install -U qiskit qiskit-machine-learning qiskit-aer qiskit-algorithms

!pip install qiskit qiskit-aer qiskit-machine-learning




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 104.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=6ad42f15745d2c324642fdfb1be1fa3d8a7053aa48bf1204c7db89cd81149f91
  Stored in directory: /root/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

QNN Regressor
✅
Flexible

In [ ]:
# qnn_quantum_pure.py
# ===========================================================
# ⚛️ QUANTUM NEURAL NETWORK REGRESSOR (QNN) 
# ===========================================================
# Universidad de Antioquia
# ===========================================================

import warnings, time, pickle, platform, os
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import trange, tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import qiskit
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator

SEED = 42
np.random.seed(SEED)

print("="*80)
print("⚛️ QNN - CUÁNTICO PURO (Parameter-shift gradients)")
print(f"Qiskit: {qiskit.__version__} | Platform: {platform.platform()}")
print("="*80, "\n")

# -------------------------
# 1) CARGA Y PREPARACIÓN DE DATOS
# -------------------------
url = "https://raw.githubusercontent.com/Alisser2001/QuantumMachineLearning/emmadev/elevators.dat"
print("Cargando dataset desde GitHub...")

try:
    import requests
    resp = requests.get(url, timeout=15)
    resp.raise_for_status()
    lines = [l.strip() for l in resp.text.splitlines()]

    attributes = [line.split()[1] for line in lines if line.lower().startswith("@attribute")]
    data_start = next(i for i, l in enumerate(lines) if l.lower() == "@data") + 1
    rows = [ln for ln in lines[data_start:] if ln and not ln.startswith('%')]
    data = [r.split(',') for r in rows]

    df = pd.DataFrame(data, columns=attributes).apply(pd.to_numeric, errors='coerce')
    df.dropna(inplace=True)
    print(f"✅ Dataset cargado: {df.shape[0]:,} filas, {df.shape[1]} columnas")

except Exception as e:
    print("❌ Error cargando dataset:", e)
    raise

# CONTROL DE TAMAÑO DE BASE DE DATOS
SAMPLE_SIZE = len(df)   # usamos todo por defecto
print(f"\n🎯 SAMPLE_SIZE = {SAMPLE_SIZE:,}")
df = df.sample(n=min(SAMPLE_SIZE, len(df)), random_state=SEED).reset_index(drop=True)

# FEATURES y TARGET
features = ['SaTime1', 'SaTime2', 'SaTime3', 'Sa']  # 4 features -> 4 qubits
target = 'Goal'

X = df[features].values.astype(float)
y = df[target].values.astype(float)

# Escalado: inputs a [0, pi], outputs a [-1,1]
scaler_X = MinMaxScaler(feature_range=(0, np.pi))
scaler_y = MinMaxScaler(feature_range=(-1, 1))

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1,1)).flatten()

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=SEED)
num_qubits = X_train.shape[1]

print(f"\nSplit: Train={X_train.shape[0]}, Test={X_test.shape[0]}, Qubits={num_qubits}\n")

# -------------------------
# 2) CONSTRUCCIÓN DEL CIRCUITO QNN (encoding + variational layers)
# -------------------------
def create_qnn_circuit(num_qubits, num_layers=2):
    """
    - input_params: serán ligados a las rotaciones de encoding (RX)
    - weight_params: parámetros entrenables (variational)
    """
    input_params = ParameterVector("x", num_qubits)
    weight_params = ParameterVector("w", num_qubits * num_layers * 2)  # 2 parámetros por qubit por layer
    qc = QuantumCircuit(num_qubits)

    # Encoding layer (feature map): RX(x_i) + RZ(x_i / 2)
    for i in range(num_qubits):
        qc.rx(input_params[i], i)
        qc.rz(input_params[i] / 2.0, i)

    # Variational layers
    idx = 0
    for layer in range(num_layers):
        for q in range(num_qubits):
            qc.ry(weight_params[idx], q); idx += 1
            qc.rz(weight_params[idx], q); idx += 1
        # entanglement ring
        for q in range(num_qubits - 1):
            qc.cz(q, q+1)
        if num_qubits > 1:
            qc.cz(num_qubits - 1, 0)
        qc.barrier()

    return qc, input_params, weight_params

NUM_LAYERS = 2
qnn_circuit, x_params, w_params = create_qnn_circuit(num_qubits, NUM_LAYERS)

print(qnn_circuit.draw())

print("Circuit created. Depth:", qnn_circuit.depth(), "Gates:", sum(qnn_circuit.count_ops().values()))
num_params = len(w_params)
print(f"Trainable params: {num_params}\n")

# Readout observable: Z on first qubit (regression value)
observable = SparsePauliOp('Z' + 'I'*(num_qubits-1))

# Estimator primitive
estimator = Estimator()

# -------------------------
# 3) FORWARD / PREDICTION
# -------------------------
def qnn_forward_single(x_value, weights):
    """Return expectation value <observable> for single sample x_value and weights."""
    bind = {}
    for i in range(num_qubits):
        bind[x_params[i]] = float(x_value[i])
    for i in range(num_params):
        bind[w_params[i]] = float(weights[i])
    bound_qc = qnn_circuit.assign_parameters(bind)
    job = estimator.run(bound_qc, observable)
    res = job.result()
    return float(res.values[0])

def qnn_forward_batch(X_batch, weights):
    return np.array([qnn_forward_single(xv, weights) for xv in X_batch], dtype=float)

# -------------------------
# 4) PARAMETER-SHIFT GRADIENT (exact for these single-qubit rotation gates)
# -------------------------
SHIFT = np.pi / 2.0  

def parameter_shift_grad_single(x_value, weights):
    """
    Compute gradient df/dw for a single input x_value using parameter-shift.
    Returns gradient vector length num_params.
    """
    grad = np.zeros_like(weights, dtype=float)
    # For each trainable parameter evaluate f(w+shift) and f(w-shift)
    for k in range(num_params):
        w_plus = weights.copy(); w_minus = weights.copy()
        w_plus[k] += SHIFT
        w_minus[k] -= SHIFT
        f_plus = qnn_forward_single(x_value, w_plus)
        f_minus = qnn_forward_single(x_value, w_minus)
        grad[k] = 0.5 * (f_plus - f_minus)  # parameter-shift rule (appropriate scaling)
    return grad

# -------------------------
# 5) LOSS and BATCH GRADIENT (MSE)
# -------------------------
def batch_loss_and_grad(X_batch, y_batch, weights):
    B = len(X_batch)
    preds = np.zeros(B, dtype=float)
    grad_total = np.zeros_like(weights, dtype=float)
    for i in range(B):
        xval = X_batch[i]
        yi = y_batch[i]
        fi = qnn_forward_single(xval, weights)
        preds[i] = fi
        dfi = parameter_shift_grad_single(xval, weights)
        grad_total += 2.0 * (fi - yi) * dfi
    grad_total /= B
    loss = float(np.mean((preds - y_batch)**2))
    return loss, grad_total

# -------------------------
# 6) OPTIMIZADOR: SGD con momentum 
# -------------------------
def train_qnn(X_train, y_train, weights_init=None,
              epochs=10, lr=0.05, batch_size=8, momentum=0.9, verbose=True):
    if weights_init is None:
        weights = np.random.uniform(-0.1, 0.1, num_params)
    else:
        weights = np.array(weights_init, dtype=float)

    velocity = np.zeros_like(weights, dtype=float)
    n = len(X_train)
    history = []
    t0 = time.time()

    for ep in range(epochs):
        perm = np.random.permutation(n)
        X_sh = X_train[perm]
        y_sh = y_train[perm]
        epoch_loss = 0.0
        for s in range(0, n, batch_size):
            xb = X_sh[s:s+batch_size]
            yb = y_sh[s:s+batch_size]
            loss, grad = batch_loss_and_grad(xb, yb, weights)
            # update with momentum
            velocity = momentum * velocity + lr * grad
            weights = weights - velocity
            epoch_loss += loss * len(xb)
        epoch_loss /= n
        history.append(epoch_loss)
        if verbose:
            print(f"Epoch {ep+1:03d}/{epochs:03d} | Loss={epoch_loss:.6f} | lr={lr:.4f} | ||w||={np.linalg.norm(weights):.4f}")
    train_time = time.time() - t0
    return weights, history, train_time

# -------------------------
# 7) EJECUTAR ENTRENAMIENTO
# -------------------------
# Inicialización y hyperparámetros
initial_weights = np.random.uniform(-0.1, 0.1, num_params)
EPOCHS = 12      # aumenta si tienes tiempo
LR = 0.06
BATCH_SIZE = 8
MOMENTUM = 0.9

print("\n🚀 Entrenando QNN (100% cuántico, parameter-shift)...")
trained_weights, history, train_time = train_qnn(
    X_train, y_train,
    weights_init=initial_weights,
    epochs=EPOCHS,
    lr=LR,
    batch_size=BATCH_SIZE,
    momentum=MOMENTUM,
    verbose=True
)
print(f"\n✅ Entrenamiento completado en {train_time:.1f}s\n")

# -------------------------
# 8) EVALUACIÓN
# -------------------------
y_pred_train_scaled = qnn_forward_batch(X_train, trained_weights)
y_pred_test_scaled  = qnn_forward_batch(X_test, trained_weights)

# Desescalar a original
y_pred_train = scaler_y.inverse_transform(y_pred_train_scaled.reshape(-1,1)).flatten()
y_pred_test  = scaler_y.inverse_transform(y_pred_test_scaled.reshape(-1,1)).flatten()
y_true_train = scaler_y.inverse_transform(y_train.reshape(-1,1)).flatten()
y_true_test  = scaler_y.inverse_transform(y_test.reshape(-1,1)).flatten()

mae_train = mean_absolute_error(y_true_train, y_pred_train)
mse_train = mean_squared_error(y_true_train, y_pred_train)
rmse_train = np.sqrt(mse_train)

mae_test = mean_absolute_error(y_true_test, y_pred_test)
mse_test = mean_squared_error(y_true_test, y_pred_test)
rmse_test = np.sqrt(mse_test)

print("\n" + "="*70)
print("RESULTADOS QNN (100% CUÁNTICO)")
print("="*70)
print(f"{'Métrica':<12}{'Train':>12}{'Test':>12}")
print("-"*70)
print(f"{'MAE':<12}{mae_train:12.6f}{mae_test:12.6f}")
print(f"{'MSE':<12}{mse_train:12.6f}{mse_test:12.6f}")
print(f"{'RMSE':<12}{rmse_train:12.6f}{rmse_test:12.6f}")
print("="*70)

# -------------------------
# 9) VISUALIZACIÓN
# -------------------------
plt.style.use('seaborn-v0_8-darkgrid')
fig, axes = plt.subplots(2, 2, figsize=(14,10))
fig.suptitle(f'QNN Cuántico Puro - {num_qubits} qubits - layers={NUM_LAYERS}', fontsize=16)

ax = axes[0,0]
ax.scatter(y_true_test, y_pred_test, alpha=0.7, s=80)
ax.plot([y_true_test.min(), y_true_test.max()], [y_true_test.min(), y_true_test.max()], 'r--')
ax.set_title(f'Test (MAE={mae_test:.4f})')

ax = axes[0,1]
ax.scatter(y_true_train, y_pred_train, alpha=0.6, s=60)
ax.plot([y_true_train.min(), y_true_train.max()], [y_true_train.min(), y_true_train.max()], 'r--')
ax.set_title(f'Train (MAE={mae_train:.4f})')

ax = axes[1,0]
ax.plot(np.arange(1, len(history)+1), history, 'o-')
ax.set_title('Training Loss (MSE)')

ax = axes[1,1]
resid = y_true_test - y_pred_test
ax.hist(resid, bins=15, edgecolor='k')
ax.set_title(f'Residuals (σ={resid.std():.4f})')

plt.tight_layout()
plt.savefig('qnn_quantum_pure_results.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Visual guardada: qnn_quantum_pure_results.png")

# -------------------------
# 10) GUARDAR MODELO
# -------------------------
model_package = {
    "trained_weights": trained_weights,
    "circuit": qnn_circuit,
    "x_params": x_params,
    "w_params": w_params,
    "scaler_X": scaler_X,
    "scaler_y": scaler_y,
    "features": features,
    "metrics": {
        "mae_test": mae_test,
        "mse_test": mse_test,
        "rmse_test": rmse_test,
        "train_time_s": train_time
    },
    "config": {
        "num_qubits": num_qubits,
        "num_layers": NUM_LAYERS,
        "num_params": num_params,
        "sample_size": len(X_train) + len(X_test)
    }
}

with open("qnn_quantum_pure_model.pkl", "wb") as f:
    pickle.dump(model_package, f)

print("\n💾 Modelo guardado: qnn_quantum_pure_model.pkl")
print("\n✨ QNN cuántico puro completado con éxito!")


⚛️ QNN - CUÁNTICO PURO (Parameter-shift gradients)
Qiskit: 1.4.5 | Platform: Linux-6.6.105+-x86_64-with-glibc2.35

Cargando dataset desde GitHub...
✅ Dataset cargado: 16,599 filas, 19 columnas

🎯 SAMPLE_SIZE = 16,599

Split: Train=13279, Test=3320, Qubits=4

     ┌──────────┐┌──────────────┐┌──────────┐┌──────────┐             ░ »
q_0: ┤ Rx(x[0]) ├┤ Rz(0.5*x[0]) ├┤ Ry(w[0]) ├┤ Rz(w[1]) ├─■────────■──░─»
     ├──────────┤├──────────────┤├──────────┤├──────────┤ │        │  ░ »
q_1: ┤ Rx(x[1]) ├┤ Rz(0.5*x[1]) ├┤ Ry(w[2]) ├┤ Rz(w[3]) ├─■──■─────┼──░─»
     ├──────────┤├──────────────┤├──────────┤├──────────┤    │     │  ░ »
q_2: ┤ Rx(x[2]) ├┤ Rz(0.5*x[2]) ├┤ Ry(w[4]) ├┤ Rz(w[5]) ├────■──■──┼──░─»
     ├──────────┤├──────────────┤├──────────┤├──────────┤       │  │  ░ »
q_3: ┤ Rx(x[3]) ├┤ Rz(0.5*x[3]) ├┤ Ry(w[6]) ├┤ Rz(w[7]) ├───────■──■──░─»
     └──────────┘└──────────────┘└──────────┘└──────────┘             ░ »
«      ┌──────────┐ ┌──────────┐             ░ 
«q_0: ─┤ Ry(w[8]) ├─┤ Rz(w[